In [ ]:
import xarray as xr
import pandas as pd
import numpy as np 
import dask as da
import shapely as shp
from scipy.interpolate import interp1d
from pathlib import Path
import rioxarray as rxr

In [ ]:
path = Path.cwd().parent / 'data' / 'in'
path

# get data

In [ ]:
data = xr.open_dataset(path / "ERA5land_global_t2m_dailyStats_mean_01Deg_2024_08_data.nc", chunks='auto', engine = 'rasterio')
data

In [ ]:
temp_data = xr.Dataset(pd.read_csv(path / "R0_pip_stats.csv")) 
temp_data

In [ ]:
data['t2m'].mean(dim='valid_time').plot(cmap='viridis')

# interpolation function

the results differ at the moment, even though they should be the same. 

In [ ]:
temps = np.arange(0., 45.1, 0.1)
valid = (~np.isnan(temp_data.Median_R0) & (temp_data.Temperature >= 0.0) & (temp_data.Temperature <= 45.1 )).values

# interp1d is deprecated, but it works and does what we need. 
# replacing this with a callback to numpy.interp is unnecessarily complicated, since 
# we need a function at the end, not the array directly.
r0_pip_fun = interp1d(
    temp_data.Temperature[valid].values, 
    temp_data.Median_R0[valid].values, 
    bounds_error=False, 
    fill_value=0.0, 
    kind='linear'
)


In [ ]:
print(f"Min: {data['t2m'].min().values}")
print(f"Max: {data['t2m'].max().values}")
print(f"Mean: {data['t2m'].mean().values}")

# apply the function to the data

In [ ]:
r0_map = xr.apply_ufunc(
    r0_pip_fun, 
    data['t2m'], 
    dask="allowed", 
    input_core_dims=[[]],
    output_core_dims=[[]],
    vectorize=True,
)
r0_map

In [ ]:
print(f"Min: {r0_map.min().values}")
print(f"Max: {r0_map.max().values}")
print(f"Mean: {r0_map.mean().values}")

In [ ]:
r0_map.mean(dim='valid_time').plot(cmap='viridis')

This looks fundamentally different from the version of R

In [ ]:
import geopandas as gpd 


def get_eurostat_geospatial(nuts_level=3, year=2024, resolution='10M'):
    """Download NUTS boundaries directly from Eurostat GISCO API"""
    
    base_url = "https://gisco-services.ec.europa.eu/distribution/v2/nuts"
    
    # Available formats: geojson, shp, topojson
    url = f"{base_url}/geojson/NUTS_RG_{resolution}_{year}_4326_LEVL_{nuts_level}.geojson"
    
    try:
        nuts_data = gpd.read_file(url)
        return nuts_data
    except Exception as e:
        print(f"Failed to download from {url}: {e}")
        return None

In [ ]:
europe_bounds = get_eurostat_geospatial(resolution = '10M',
                                    nuts_level = 3, 
                                    year = 2024) 

In [ ]:
europe_bounds

In [ ]:
minlong, minlat, maxlong, maxlat = europe_bounds.total_bounds

In [ ]:
print(f"Europe bounds: {minlong}, {minlat}, {maxlong}, {maxlat}")

this doesn´t work yet. 

In [ ]:
r0_map_europe = r0_map.sel(
    longitude=slice(minlong, maxlong),  # min_x to max_x
    latitude=slice(minlat, maxlat)    # max_y to min_y (latitude might be descending)
)

In [ ]:
r0_map_europe

In [ ]:
r0_map_europe.mean(dim='valid_time').plot(cmap='viridis')